<a href="https://colab.research.google.com/github/Tannazsetayesh/ai-chatbot-rag/blob/main/Anika_IRS_AI_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Initial Setup**

## Install dependnecies

In [2]:
!pip install google-cloud-storage pinecone-client tqdm pandas pinecone google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.3/524.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 12.4 MB/s eta 0:00:00


## GCP Permissions

In [ ]:
!gcloud projects add-iam-policy-binding vertex-ai-rag-463204 --member="user:tanaz.setayesh@gmail.com" --role="roles/aiplatform.user"
!gcloud projects add-iam-policy-binding vertex-ai-rag-463204 --member="user:tanaz.setayesh@gmail.com" --role="roles/aiplatform.viewer"

## Import dependencies

In [25]:
import json
import os
import requests
import time
import uuid
import re

from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from datetime import datetime
from pinecone import Pinecone
from tqdm.notebook import tqdm

from google.colab import auth
from google.cloud import aiplatform
from google.cloud import storage
from google.cloud.aiplatform_v1 import PredictionServiceClient
from google.protobuf.struct_pb2 import Struct
from google.api_core.exceptions import ResourceExhausted, NotFound, PermissionDenied

from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput
from vertexai.generative_models import GenerativeModel

In [23]:


# Google Cloud Storage configuration
BUCKET_NAME = "anika-chatbot"
GCS_PREFIX = "documents_"

# Authenticate in Colab
auth.authenticate_user()
print('GCP user is Authenticated.')


GCP user is Authenticated.


# **Read user data**

## Select & Acquire The Dataset

In [ ]:
def download_html_to_gcs(start_url="https://www.irs.gov/irm/part1", bucket_name="anika-chatbot"):

    print(f"Starting download from: {start_url}")
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    start_url_parsed = urlparse(start_url)
    start_url_base = start_url_parsed.scheme + "://" + start_url_parsed.netloc + start_url_parsed.path

    def get_html(url):
        """Fetches HTML content from a URL."""
        try:
            print(f"Fetching URL: {url}")
            response = requests.get(url, timeout=10) # Added timeout
            response.raise_for_status()  # Raise an exception for bad status codes
            return response.text
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")
            return None

    def upload_to_gcs(html_content, blob_name):
        """Uploads HTML content to GCS."""
        try:
            print(f"Uploading to GCS blob: {blob_name}")
            blob = bucket.blob(blob_name)
            blob.upload_from_string(html_content, content_type='text/html')
            print(f"Successfully uploaded {blob_name}")
            return True # Indicate successful upload
        except Exception as e:
            print(f"Error uploading to GCS blob {blob_name}: {e}")
            return False # Indicate failed upload


    visited_urls = set()
    # Remove fragment from start_url before adding to queue and visited set
    urls_to_visit = [(start_url_base, 0)] # Store (url, level)
    visited_urls.add(start_url_base)

    files_downloaded_count = 0 # Initialize counter

    while urls_to_visit and files_downloaded_count < 100: # Add condition for file count
        current_url, level = urls_to_visit.pop(0) # Use pop(0) for breadth-first

        # No need to check visited_urls here as we add to it when adding to urls_to_visit
        if level > 2:
            continue

        html_content = get_html(current_url)
        if html_content:
            # Create a blob name from the URL
            blob_name = current_url.replace("https://", "").replace("http://", "").replace("/", "_") + ".html"
            if upload_to_gcs(html_content, blob_name): # Check if upload was successful
                files_downloaded_count += 1 # Increment counter only on successful upload

            if level < 2 and files_downloaded_count < 100: # Add condition for file count
                soup = BeautifulSoup(html_content, 'html.parser')
                for link in soup.find_all('a', href=True):
                    href = link.get('href')
                    if href:
                        # Resolve relative URLs
                        new_url = urljoin(current_url, href)

                        # Remove fragment from the new_url for comparison and storage
                        new_url_parsed = urlparse(new_url)
                        new_url_base = new_url_parsed.scheme + "://" + new_url_parsed.netloc + new_url_parsed.path

                        # Check if the new URL is within the same domain AND is a subpath of the start_url
                        if urlparse(new_url_base).netloc == urlparse(start_url_base).netloc and new_url_base.startswith(start_url_base):
                            if new_url_base not in visited_urls:
                                visited_urls.add(new_url_base) # Add to visited set
                                urls_to_visit.append((new_url_base, level + 1))
                                print(f"Adding {new_url_base} to visit queue at level {level + 1}")

        if files_downloaded_count >= 100:
            print("Reached maximum number of files to download (100). Stopping.")
            break # Exit the loop if max files downloaded

        time.sleep(1) # Add a small delay to avoid overwhelming servers



## Prepare & Extract Text

In [6]:
# Read all HTML fiels and create single json file with Title and Text
def process_html_from_gcs(bucket_name="anika-chatbot"):

    print(f"Processing HTML files from bucket: {bucket_name}")

    client = storage.Client()
    bucket = client.get_bucket(bucket_name)

    # List of tags to remove
    tags_to_remove = [
        "script",
        "style",
        "iframe",
        "noscript",
        "header",
        "footer",
        "nav",
        "link",
        "meta",
        "form",
        "input",
        "button",
        "select",
        "svg",
    ]

    documents = []

    # List blobs in the bucket that end with .html
    blobs = bucket.list_blobs(prefix="") # List all blobs, we'll filter later

    for blob in blobs:
        if blob.name.endswith(".html"):
            print(f"Reading file: {blob.name}")
            try:
                # Download the blob content as a string
                html_content = blob.download_as_text()

                # Parse the HTML content
                soup = BeautifulSoup(html_content, 'html.parser')

                # Remove specified tags
                for tag in tags_to_remove:
                    for element in soup.find_all(tag):
                        element.extract()

                # Extract text from the remaining content
                text = soup.get_text(separator='\n', strip=True)

                # Get the title (if available in the HTML)
                title = soup.title.string if soup.title else blob.name.replace(".html", "").replace("_", " ")

                documents.append({'title': title, 'text': text})

            except Exception as e:
                print(f"Error processing file {blob.name}: {e}")

    # Write the documents data structure to a JSON file in GCS
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    output_blob_name = f"documents_{timestamp}.json"

    try:
        output_blob = bucket.blob(output_blob_name)
        output_blob.upload_from_string(json.dumps(documents), content_type='application/json')
        print(f"Successfully wrote extracted data to {output_blob_name} in bucket {bucket_name}")
    except Exception as e:
        print(f"Error writing documents to GCS {output_blob_name}: {e}")

process_html_from_gcs()

Processing HTML files from bucket: anika-chatbot
Reading file: gui/index.html
Reading file: www.irs.gov_irm_part1.html
Reading file: www.irs.gov_irm_part1_irm_01-001-006.html
Reading file: www.irs.gov_irm_part1_irm_01-001-007.html
Reading file: www.irs.gov_irm_part1_irm_01-001-011.html
Reading file: www.irs.gov_irm_part1_irm_01-001-012.html
Reading file: www.irs.gov_irm_part1_irm_01-001-016.html
Reading file: www.irs.gov_irm_part1_irm_01-001-017.html
Reading file: www.irs.gov_irm_part1_irm_01-001-018.html
Reading file: www.irs.gov_irm_part1_irm_01-001-020.html
Reading file: www.irs.gov_irm_part1_irm_01-001-021.html
Reading file: www.irs.gov_irm_part1_irm_01-001-026.html
Reading file: www.irs.gov_irm_part1_irm_01-001-027.html
Reading file: www.irs.gov_irm_part1_irm_01-001-028.html
Reading file: www.irs.gov_irm_part1_irm_01-001-029.html
Reading file: www.irs.gov_irm_part1_irm_01-001-031.html
Reading file: www.irs.gov_irm_part1_irm_01-001-033.html
Reading file: www.irs.gov_irm_part1_irm_0

# **Embedding & Indexing**

## Pincone initializing

In [ ]:


# Pinecone configuration
PINECONE_API_KEY = "pcsk_5Npvnn_PXGZ62e3iD3aRdS54kiezL4D6a7qfVNGUUDxLbLQBBB5h7RvLVEya2ZsjszFDbe"
PINECONE_HOST = "https://irs-chatbot-index-3c8vbcs.svc.aped-4627-b74a.pinecone.io"
NAMESPACE = "irs-namespace"

# Initialize Pinecone client
try:
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index = pc.Index(host=PINECONE_HOST)
    print("Pinecone client initialized successfully")
except Exception as e:
    print(f"Failed to initialize Pinecone client: {e}")
    raise

Pinecone client initialized successfully


## Load Cleaned Documents from Cloud Storage

In [11]:
# Define function to read JSON from GCS
def read_json_from_gcs(bucket_name=BUCKET_NAME, prefix=GCS_PREFIX):
    """
    Read JSON files from GCS bucket that match the given prefix.
    Returns a list of JSON objects.
    """
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blobs = bucket.list_blobs(prefix=prefix)

        all_documents = []
        for blob in tqdm(list(blobs), desc="Reading GCS files"):
            if blob.name.endswith(".json"):
                print(f"Processing file: {blob.name}")
                json_data = blob.download_as_text()
                documents = json.loads(json_data)
                all_documents.extend(documents)

        print(f"Retrieved {len(all_documents)} documents from GCS")
        return all_documents
    except Exception as e:
        print(f"Error reading from GCS: {e}")
        raise

#temp=read_json_from_gcs()

Reading GCS files:   0%|          | 0/1 [00:00<?, ?it/s]

Processing file: documents_20250620200924.json
Retrieved 101 documents from GCS


## Chunking Text for Embedding

In [16]:
# Text chunking function
def chunk_text(text, max_chunk_size=500):
    """
    Split text into chunks of approximately max_chunk_size characters.
    Attempts to split at sentence boundaries to preserve context.
    """
    try:
        sentences = re.split(r'(?<=[.!?])\s+', text)
        chunks = []
        current_chunk = ""
        for sentence in sentences:
            if len(current_chunk) + len(sentence) <= max_chunk_size:
                current_chunk += sentence + " "
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence + " "
        if current_chunk:
            chunks.append(current_chunk.strip())
        print(f"Chunked text into {len(chunks)} chunks")
        return chunks
    except Exception as e:
        print(f"Error chunking text: {e}")
        return [text]  # Fallback to single chunk if error occurs

## Prepare Pinecone records

In [18]:
# Define function to prepare Pinecone records
def prepare_records(documents):
    """
    Prepare records for Pinecone upsert from JSON documents.
    Chunks large texts and generates unique IDs.
    Ensures records have a valid 'text' field and metadata fields are flat and of valid types.
    """
    try:
        records = []
        for doc in tqdm(documents, desc="Preparing records"):
            title = doc.get("title", "")
            text_content = doc.get("text", "")

            # Skip documents with empty text
            if not text_content.strip():
                print(f"Skipping document with empty text: title='{title}'")
                continue

            # Chunk the text if it's too long
            chunks = chunk_text(text_content, max_chunk_size=500)

            for i, chunk in enumerate(chunks):
                # Validate chunk is not empty
                if not chunk.strip():
                    print(f"Skipping empty chunk {i} for document: title='{title}'")
                    continue

                # Prepare metadata fields (flattened, no nested dictionary)
                metadata = {
                    "title": str(title),  # Ensure string type
                    "chunk_index": i,  # Integer is valid
                }

                # Validate metadata types
                for key, value in metadata.items():
                    if not isinstance(value, (str, int, float, bool, list)):
                        print(f"Invalid metadata type for {key}: {type(value)}. Converting to string.")
                        metadata[key] = str(value)
                    if isinstance(value, list) and not all(isinstance(item, str) for item in value):
                        print(f"Invalid list type for {key}. Converting to string.")
                        metadata[key] = str(value)

                record = {
                    "_id": str(uuid.uuid4()),  # Unique ID for each chunk
                    "text": chunk,  # Text field for embedding
                    "category": "irs_info",  # Customize as needed
                    **metadata  # Flatten metadata into top-level fields
                }
                records.append(record)

        if records:
            print(f"Sample record: {records[0]}")

        print(f"Prepared {len(records)} valid records for upsert (skipped {len(documents) * len(chunks) - len(records)} invalid chunks)")
        return records
    except Exception as e:
        print(f"Error preparing records: {e}")
        raise

## Upsert (insert/update) to pinecone


In [20]:
# Define function to upsert to Pinecone

def upsert_to_pinecone(records, namespace, tokens_per_record=250, token_limit_per_minute=250000):
    """
    Upsert records to Pinecone index in batches with rate-limiting.
    Validates records before upserting and logs contents for debugging if an error occurs.
    Model: llama-text-embed-v2
    Args:
        records: List of records to upsert.
        namespace: Pinecone namespace.
        tokens_per_record: Estimated tokens per record (default: 250, ~500 characters).
        token_limit_per_minute: Token limit per minute (default: 250,000).
    """
    try:
        # Validate records before upserting
        for record in records:
            if "text" not in record:
                print(f"Invalid record missing 'text' field: {record}")
                raise ValueError(f"Record missing 'text' field: {record}")
            if not isinstance(record.get("text"), str):
                print(f"Invalid 'text' type in record: {record}")
                raise ValueError(f"Invalid 'text' type: {type(record.get('text'))}")
            # Check metadata fields
            for key, value in record.items():
                if key not in ("_id", "text"):
                    if not isinstance(value, (str, int, float, bool, list)):
                        print(f"Invalid type for {key}: {type(value)} in record: {record}")
                        raise ValueError(f"Invalid type for {key}: {type(value)}")
                    if isinstance(value, list) and not all(isinstance(item, str) for item in value):
                        print(f"Invalid list type for {key}: {value} in record: {record}")
                        raise ValueError(f"Invalid list type for {key}: {value}")

        batch_size = 50  # Pinecone recommends batching for efficiency
        max_records_per_minute = token_limit_per_minute // tokens_per_record  # Max records per minute
        records_per_second = max_records_per_minute / 60  # Spread over 60 seconds
        seconds_per_batch = batch_size / records_per_second if records_per_second > 0 else 1  # Delay per batch

        print(f"Rate-limiting: ~{tokens_per_record} tokens/record, {token_limit_per_minute} tokens/minute, "
                     f"{max_records_per_minute} records/minute, {seconds_per_batch:.2f} seconds/batch")

        total_tokens = 0
        for i in tqdm(range(0, len(records), batch_size), desc="Upserting to Pinecone"):
            batch = records[i:i + batch_size]
            batch_tokens = len(batch) * tokens_per_record
            total_tokens += batch_tokens

            try:
                start_time = time.time()
                index.upsert_records(namespace=namespace, records=batch)
                elapsed_time = time.time() - start_time
                print(f"Upserted batch {i // batch_size + 1} with {len(batch)} records "
                             f"(~{batch_tokens} tokens, total ~{total_tokens} tokens, took {elapsed_time:.2f}s)")

                # Add delay to stay within rate limit
                if i + batch_size < len(records):  # Skip delay for last batch
                    sleep_time = max(0, seconds_per_batch - elapsed_time)
                    if sleep_time > 0:
                        print(f"Sleeping for {sleep_time:.2f} seconds to respect rate limit")
                        time.sleep(sleep_time)
            except Exception as batch_error:
                print(f"Error upserting batch {i // batch_size + 1}: {batch_error}")
                # Log the first few records in the batch for debugging
                for record in batch[:5]:  # Limit to 5 to avoid flooding logs
                    print(f"Problematic record: {record}")
                raise
        print(f"Successfully upserted {len(records)} records into Pinecone index")

    except Exception as e:
        print(f"Error upserting to Pinecone: {e}")
        raise

# **Orchestrate the Full Upsert Pipeline**

In [19]:

def digest_insert():
    try:
        # download HTML files to gcs bucket
        download_html_to_gcs()

        # Read JSON files from GCS
        documents = read_json_from_gcs(BUCKET_NAME, GCS_PREFIX)

        # Prepare records for upsert
        records = prepare_records(documents)

        # Upsert records to Pinecone
        upsert_to_pinecone(records, NAMESPACE, tokens_per_record=250)

    except Exception as e:
        print(f"Pipeline failed: {e}")
        raise

# digest_insert()

Reading GCS files:   0%|          | 0/1 [00:00<?, ?it/s]

Processing file: documents_20250620200924.json
Retrieved 101 documents from GCS


Preparing records:   0%|          | 0/101 [00:00<?, ?it/s]

Chunked text into 1 chunks
Chunked text into 2 chunks
Chunked text into 95 chunks
Chunked text into 108 chunks
Chunked text into 57 chunks
Chunked text into 211 chunks
Chunked text into 503 chunks
Chunked text into 56 chunks
Chunked text into 61 chunks
Chunked text into 44 chunks
Chunked text into 115 chunks
Chunked text into 35 chunks
Chunked text into 110 chunks
Chunked text into 34 chunks
Chunked text into 52 chunks
Chunked text into 19 chunks
Chunked text into 13 chunks
Chunked text into 57 chunks
Chunked text into 74 chunks
Chunked text into 241 chunks
Chunked text into 138 chunks
Chunked text into 11 chunks
Chunked text into 101 chunks
Chunked text into 169 chunks
Chunked text into 42 chunks
Chunked text into 541 chunks
Chunked text into 308 chunks
Chunked text into 743 chunks
Chunked text into 462 chunks
Chunked text into 305 chunks
Chunked text into 287 chunks
Chunked text into 231 chunks
Chunked text into 59 chunks
Chunked text into 295 chunks
Chunked text into 261 chunks
Chun

# AI / Vector Services

## Initiliaze pinecone and vertex ai

In [24]:

# Vertex AI configuration
PROJECT_ID = "vertex-ai-rag-463204"
LOCATION = "us-central1"
EMBEDDING_MODEL = "text-embedding-005"  # Confirmed accessible
FALLBACK_EMBEDDING_MODEL = "text-embedding-004"  # Confirmed accessible
LLM_MODEL = "gemini-2.0-flash"  # Primary LLM
FALLBACK_LLM_MODEL = "text-bison"  # Fallback using Predict API

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# Pinecone configuration
PINECONE_API_KEY = "pcsk_5Npvnn_PXGZ62e3iD3aRdS54kiezL4D6a7qfVNGUUDxLbLQBBB5h7RvLVEya2ZsjszFDbe"
PINECONE_HOST = "https://irs-chatbot-index-3c8vbcs.svc.aped-4627-b74a.pinecone.io"
NAMESPACE = "irs-namespace"

# Initialize Pinecone client
try:
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index = pc.Index(host=PINECONE_HOST)
    print("Pinecone client initialized successfully")
    print(index.describe_index_stats())
except Exception as e:
    print(f"Failed to initialize Pinecone client: {e}")
    raise

# Initialize Vertex AI for both embedding and generative models

try:
    aiplatform.init(project=PROJECT_ID, location=LOCATION)
    import vertexai
    vertexai.init(project=PROJECT_ID, location=LOCATION)  # For Gemini
    print("Vertex AI initialized successfully")
except Exception as e:
    print(f"Failed to initialize Vertex AI: {e}")
    raise

Pinecone client initialized successfully
{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 400},
                'irs-namespace': {'vector_count': 50439}},
 'total_vector_count': 50839,
 'vector_type': 'dense'}
Vertex AI initialized successfully


## Generate Embeddings for a User Query

In [29]:
# Embed user query with Vertex AI
def embed_query(query, project_id, location, model_name, fallback_model="text-embedding-004", task_type="RETRIEVAL_QUERY"):
    try:
        start_time = time.time()
        for model in [model_name, fallback_model]:
            try:
                embedding_model = TextEmbeddingModel.from_pretrained(model)
                embedding_input = TextEmbeddingInput(text=query, task_type=task_type)
                for attempt in range(3):
                    try:
                        response = embedding_model.get_embeddings([embedding_input])
                        embedding = response[0].values
                        if len(embedding) != 768:
                            print(f"Invalid embedding dimension for model {model}: {len(embedding)}")
                            raise ValueError(f"Expected 768 dimensions, got {len(embedding)}")
                        elapsed_time = time.time() - start_time
                        print(f"Embedded query '{query[:50]}...' with {model} in {elapsed_time:.2f}s")
                        return embedding
                    except ResourceExhausted as e:
                        wait_time = (2 ** attempt) * 10
                        print(f"Vertex AI rate limit hit for {model}: {e}. Waiting {wait_time}s.")
                        time.sleep(wait_time)
                        if attempt == 2:
                            raise
                    except Exception as e:
                        print(f"Error embedding query with {model}: {e}")
                        if model == fallback_model:
                            raise
                        break
            except Exception as e:
                print(f"Failed to initialize model {model}: {e}")
                if model == fallback_model:
                    raise
                continue
        raise ValueError(f"Both models {model_name} and {fallback_model} failed")
    except Exception as e:
        print(f"Failed to embed query: {e}")
        raise

## Retrieve Top Matches

In [28]:
# Search in Pinecone index
def search_pinecone(query_embedding, namespace, top_k=5):
    try:
        start_time = time.time()
        response = index.query(
            vector=query_embedding,
            top_k=top_k,
            namespace=namespace,
            include_metadata=True,
            include_values=False
        )
        matches = response.get("matches", [])
        results = [
            {
                "id": match["id"],
                "text": match["metadata"].get("text", ""),
                "metadata": {k: v for k, v in match["metadata"].items() if k != "text"},
                "score": match["score"]
            }
            for match in matches
        ]
        elapsed_time = time.time() - start_time
        print(f"Retrieved {len(results)} chunks in {elapsed_time:.2f}s")
        return results
    except Exception as e:
        print(f"Error searching Pinecone: {e}")
        raise

## Generate Answers with Vertex AI LLM

In [43]:
# Generate answer with Vertex AI LLM

def generate_answer(query, chunks, project_id, location, model_name, fallback_model="text-bison", max_tokens=512):
    """
    Generate answer using Vertex AI LLM (Gemini Flash 2.0).
    Args:
        query: User query.
        chunks: Retrieved Pinecone chunks.
        project_id: Google Cloud project ID.
        location: Vertex AI region.
        model_name: LLM model (e.g., gemini-2.0-flash).
        fallback_model: Fallback LLM (e.g., text-bison).
        max_tokens: Maximum output tokens.
    Returns:
        Generated answer string.
    """
    try:
        context = "\n\n".join([
            f"Chunk {i+1} (Source: {chunk['metadata'].get('source_file', 'unknown')}, "
            f"Title: {chunk['metadata'].get('title', 'unknown')}):\n{chunk['text']}"
            for i, chunk in enumerate(chunks)
        ])

        # print(f"context: {context}")

        prompt = (
            "You are a helpful IRS tax assistant. Based on the provided context, answer the user's query accurately and concisely. "
            "If the context doesn't contain relevant information, say so and provide a general response if possible.\n\n"
            f"Context:\n{context}\n\n"
            f"Query: {query}\n\n"
            "Answer:"
        )

        # Try primary model (Gemini)
        if model_name.startswith("gemini"):
            try:
                start_time = time.time()
                model = GenerativeModel(model_name)
                for attempt in range(3):
                    try:
                        response = model.generate_content(
                            prompt,
                            generation_config={
                                "max_output_tokens": max_tokens,
                                "temperature": 0.7,
                                "top_p": 0.9
                            }
                        )
                        answer = response.text.strip()
                        elapsed_time = time.time() - start_time
                        print(f"Generated answer with {model_name} in {elapsed_time:.2f}s")
                        return answer
                    except ResourceExhausted as e:
                        wait_time = (2 ** attempt) * 10
                        print(f"Vertex AI rate limit hit for {model_name}: {e}. Waiting {wait_time}s.")
                        time.sleep(wait_time)
                        if attempt == 2:
                            raise
                    except Exception as e:
                        print(f"Error generating answer with {model_name}: {e}")
                        break  # Try fallback
            except Exception as e:
                print(f"Failed to initialize {model_name}: {e}")

        # Fallback to text-bison using Predict API
        try:
            client = PredictionServiceClient()
            endpoint = f"projects/{project_id}/locations/{location}/publishers/google/models/{fallback_model}"
            instance = Struct()
            instance["content"] = prompt
            start_time = time.time()
            for attempt in range(3):
                try:
                    response = client.predict(
                        endpoint=endpoint,
                        instances=[instance],
                        parameters={
                            "maxOutputTokens": max_tokens,
                            "temperature": 0.6,
                            "topP": 0.7
                        }
                    )
                    answer = response.predictions[0]["content"].strip()
                    elapsed_time = time.time() - start_time
                    print(f"Generated answer with {fallback_model} in {elapsed_time:.2f}s")
                    return answer
                except ResourceExhausted as e:
                    wait_time = (2 ** attempt) * 10
                    print(f"Vertex AI rate limit hit for {fallback_model}: {e}. Waiting {wait_time}s.")
                    time.sleep(wait_time)
                    if attempt == 2:
                        raise
                except Exception as e:
                    print(f"Error generating answer with {fallback_model}: {e}")
                    raise
        except Exception as e:
            print(f"Failed to initialize {fallback_model}: {e}")
            raise ValueError(f"Both models {model_name} and {fallback_model} failed")

    except Exception as e:
        print(f"Failed to generate answer: {e}")
        raise

## Orchestrate the RAG Chatbot

In [42]:
# RAG chatbot function
def rag_chatbot(query):
    print (f"Sending user query ({query}) to RAG...")
    try:
        print (f"Embeding user query...")
        query_embedding = embed_query(query, PROJECT_ID, LOCATION, EMBEDDING_MODEL, FALLBACK_EMBEDDING_MODEL)

        print (f"Search in pinecone...")
        chunks = search_pinecone(query_embedding, NAMESPACE, top_k=10)

        print (f"Generating Answer...")
        answer = generate_answer(query, chunks, PROJECT_ID, LOCATION, LLM_MODEL, FALLBACK_LLM_MODEL)

        print(f"Input Query: {query}\nFinal Answer: {answer}")
        return answer
    except Exception as e:
        print(f"RAG pipeline failed: {e}")
        raise

In [46]:
# Test the RAG chatbot
def test_rag_chatbot():

    test_queries = [
        "What do you know?",
        "What is responsibilities of an SB/SE Field?",
        "What is Internal Reviews staff responsibilities?",
        "what is Chief Counsel coordinates?"
    ]

    for query in test_queries:
        print(f"\nQuery: {query}")
        try:
            answer = rag_chatbot(query)
            print(f"Answer: {answer}")
        except Exception as e:
            print(f"Error: {e}")

test_rag_chatbot()


Query: What do you know?
Sending user query (What do you know?) to RAG...
Embeding user query...
Embedded query 'What do you know?...' with text-embedding-005 in 0.49s
Search in pinecone...
Retrieved 10 chunks in 0.06s
Generating Answer...
Generated answer with gemini-2.0-flash in 2.17s
Input Query: What do you know?
Final Answer: Based on the provided documents, I have information regarding the roles and responsibilities of an SB/SE Field and Office Examination Group Manager and some information on insolvency cases.

Specifically, regarding Group Managers:

*   They should keep the territory manager and the group informed of significant matters.
*   They are expected to be flexible as priorities change.
*   They should inform the territory manager of the impact of shifting priorities on resources and programs.
*   They are expected to be team players, potentially sacrificing group goals for broader organizational goals.
*   They receive Area goals and priorities from Territory manage

## Interactive Chatbot UI

In [47]:
# Interactive chatbot
import ipywidgets as widgets
from IPython.display import display

query_input = widgets.Text(description="Query:", layout={'width': '500px'})
output = widgets.Output()
button = widgets.Button(description="Submit")

def on_button_click(b):
    with output:
        output.clear_output()
        query = query_input.value
        if query.strip():
            print(f"Query: {query}")
            answer = rag_chatbot(query)
            print(f"Answer: {answer}")
        else:
            print("Please enter a query.")

button.on_click(on_button_click)
display(query_input, button, output)

Text(value='', description='Query:', layout=Layout(width='500px'))

Button(description='Submit', style=ButtonStyle())

Output()